First, setup the environment so that we have graphics

In [ ]:
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = (12, 8)  # Figure size for inline display
import os

Import pynx classes

In [ ]:
from pynx.ptycho.runner.id13 import PtychoRunnerScanID13, params
print('Import OK')

First, make sure all results will be saved in **your sub-directory**, so that all results don't collide !

In [ ]:
params['saveprefix']='vincent/ResultsScan%04d/Run%04d'

Set the parameters for the optimization.
In this case the scan is fast (50ms/point), so intensity is low - crop the data to 256 pixels to save memory.

**NOTE**: there are 61\*61 images of 2070x2167 Mbytes, so this requires **64 Gb** for the raw data.

In [ ]:
params['specfile']='/home/pynx/UM2017/20161021id13-siemens2/siemens2.dat'
params['scan']=13
params['h5file']='/home/pynx/UM2017/20161021id13-siemens2/pst_fix_he_198_master.h5'
params['probe']='40e-6x40e-6,0.01'  # Starting from a simulated probe, here described as gaussian
params['defocus']=-100e-6        #Defocus the calculated initial probe by 100 microns
params['detectordistance']=1.7598
params['ptychomotors']='nnp2,nnp3,-x,y'   # id13 motors
params['algorithm'] = '20DM'    # Begin with difference map, object only
params['object'] = 'random,0.95,1,0,0.2'   # High energy => start from an almost-flat object
params['verbose'] = 10          # Print every 10 cycles
params['liveplot'] = True       # Liveplot updated at the end of each cell.
#params['moduloframe'] =2,0      # Take only half frames (faster, less memory used)
params['maxsize'] = 256         # Use only 256 pixels (faster, less memory used)
params['roi'] = 1131,1387,861,1117       # xmin, xmax, ymin, ymax. Data is partially corrupted so use manual ROI

In [ ]:
ws = PtychoRunnerScanID13(params, params['scan'])

Load all frames from the Eiger HDF5 data. ***There are 3721 frames so this takes a while !***

In [ ]:
ws.load_data()

In [ ]:
ws.center_crop_data()  # Crop raw data
ws.prepare()  # This will prepare the initial object and probe

In [ ]:
ws.run()  # Run the optimization,as defined in params['algorithm']

In [ ]:
ws.run_algorithm('probe=1,40DM')  # Activate probe optimization, continue difference map

In [ ]:
ws.run_algorithm('50AP')

In [ ]:
ws.run_algorithm('50ML')

In [ ]:
ws.run_algorithm('Analyze')  # Analyze the optimized probe (modes, propagation)

### Restart using the found probe
Restart with the same probe, and a random object

In [ ]:
params['loadprobe'] = os.path.join(os.path.dirname(params['saveprefix']%(params['scan'], 0)),'latest.npz')
ws.prepare()
ws.run()

In [ ]:
ws.run_algorithm('probe=1,80DM')

In [ ]:
ws.run_algorithm('100AP')

In [ ]:
ws.run_algorithm('100ML')

In [ ]:
ws.run_algorithm('Analyze')  # Analyze the optimized probe (modes, propagation)

In [ ]:
ws.run_algorithm('nbprobe=3,100AP')

In [ ]:
ws.run_algorithm('100ML')

In [ ]:
ws.save_plot(ws._run, display_plot=True) # This makes a more elaborate plot than 'liveplot'

In [ ]:
ws.run_algorithm('Analyze')  # Analyze the optimized probe (modes, propagation)

In [ ]:
ws.plot_llk_history()